In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.layered.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.lqns.*
import jline.util.matrix.*

In [ ]:
val model = LayeredNetwork("myLayeredModel")

// Definition of processors - matching MATLAB exactly
val P = mutableListOf<Processor>()
P.add(Processor(model, "R1_Processor", 100, SchedStrategy.FCFS))     // P{1}
P.add(Processor(model, "R2_Processor", GlobalConstants.MaxInt, SchedStrategy.INF))  // P{2} - Inf as large int
P.add(Processor(model, "R3_Processor", 2, SchedStrategy.FCFS))       // P{3}
P.add(Processor(model, "R1A_Processor", 7, SchedStrategy.FCFS))      // P{4}
P.add(Processor(model, "R1B_Processor", 3, SchedStrategy.FCFS))      // P{5}
P.add(Processor(model, "R2A_Processor", 4, SchedStrategy.FCFS))      // P{6}
P.add(Processor(model, "R2B_Processor", 5, SchedStrategy.FCFS))      // P{7}

In [ ]:
// Definition of tasks - matching MATLAB exactly
val T = mutableListOf<Task>()
T.add(Task(model, "R1_Task", 100, SchedStrategy.REF).on(P[0]).setThinkTime(Exp.fitMean(20.0)))  // T{1}
T.add(Task(model, "R2_Task", GlobalConstants.MaxInt, SchedStrategy.INF).on(P[1]).setThinkTime(Immediate()))  // T{2}
T.add(Task(model, "R3_Task", 2, SchedStrategy.FCFS).on(P[2]).setThinkTime(Immediate()))       // T{3}
T.add(Task(model, "R1A_Task", 7, SchedStrategy.FCFS).on(P[3]).setThinkTime(Immediate()))      // T{4}
T.add(Task(model, "R1B_Task", 3, SchedStrategy.FCFS).on(P[4]).setThinkTime(Immediate()))      // T{5}
T.add(Task(model, "R2A_Task", 4, SchedStrategy.FCFS).on(P[5]).setThinkTime(Immediate()))      // T{6}
T.add(Task(model, "R2B_Task", 5, SchedStrategy.FCFS).on(P[6]).setThinkTime(Immediate()))      // T{7}

In [ ]:
// Definition of entries - matching MATLAB exactly
val E = mutableListOf<Entry>()
E.add(Entry(model, "R1_Ref_Entry").on(T[0]))          // E{1}
E.add(Entry(model, "R2_Synch_A2_Entry").on(T[1]))     // E{2}
E.add(Entry(model, "R2_Synch_A5_Entry").on(T[1]))     // E{3}
E.add(Entry(model, "R3_Synch_A9_Entry").on(T[2]))     // E{4}
E.add(Entry(model, "R1A_Synch_A1_Entry").on(T[3]))    // E{5}
E.add(Entry(model, "R1A_Synch_A2_Entry").on(T[3]))    // E{6}
E.add(Entry(model, "R1A_Synch_A3_Entry").on(T[3]))    // E{7}
E.add(Entry(model, "R1B_Synch_A4_Entry").on(T[4]))    // E{8}
E.add(Entry(model, "R1B_Synch_A5_Entry").on(T[4]))    // E{9}
E.add(Entry(model, "R1B_Synch_A6_Entry").on(T[4]))    // E{10}
E.add(Entry(model, "R2A_Synch_A7_Entry").on(T[5]))    // E{11}
E.add(Entry(model, "R2A_Synch_A8_Entry").on(T[5]))    // E{12}
E.add(Entry(model, "R2A_Synch_A11_Entry").on(T[5]))   // E{13}
E.add(Entry(model, "R2B_Synch_A9_Entry").on(T[6]))    // E{14}
E.add(Entry(model, "R2B_Synch_A10_Entry").on(T[6]))   // E{15}
E.add(Entry(model, "R2B_Synch_A12_Entry").on(T[6]))   // E{16}

In [ ]:
// Definition of activities - matching MATLAB exactly (all 29 activities)
val A = mutableListOf<Activity>()

// T1 activities (A{1}-A{6} in MATLAB)
A.add(Activity(model, "A1_Empty", Immediate()).on(T[0]).boundTo(E[0]).synchCall(E[4], 1))  // A{1}
A.add(Activity(model, "A2_Empty", Immediate()).on(T[0]).synchCall(E[5], 1))  // A{2}
A.add(Activity(model, "A5_Empty", Immediate()).on(T[0]).synchCall(E[8], 1))  // A{3}
A.add(Activity(model, "A6_Empty", Immediate()).on(T[0]).synchCall(E[9], 1))  // A{4}
A.add(Activity(model, "A3_Empty", Immediate()).on(T[0]).synchCall(E[6], 1))  // A{5}
A.add(Activity(model, "A4_Empty", Immediate()).on(T[0]).synchCall(E[7], 1))  // A{6}

// T2 activities (A{7}-A{13} in MATLAB)
A.add(Activity(model, "E4_Empty", Immediate()).on(T[1]).boundTo(E[1]))  // A{7}
A.add(Activity(model, "A7_Empty", Immediate()).on(T[1]).synchCall(E[10], 1))  // A{8}
A.add(Activity(model, "A8_Empty", Immediate()).on(T[1]).synchCall(E[11], 1))  // A{9}
A.add(Activity(model, "A9_Empty", Immediate()).on(T[1]).synchCall(E[13], 1))  // A{10}
A.add(Activity(model, "A11_Empty", Immediate()).on(T[1]).synchCall(E[12], 1).repliesTo(E[1]))  // A{11}
A.add(Activity(model, "A12_Empty", Immediate()).on(T[1]).boundTo(E[2]).synchCall(E[15], 1).repliesTo(E[2]))  // A{12}
A.add(Activity(model, "A10_Empty", Immediate()).on(T[1]).synchCall(E[14], 1))  // A{13}

// T3 activity (A{14} in MATLAB)
A.add(Activity(model, "A13", Exp.fitMean(10.0)).on(T[2]).boundTo(E[3]).repliesTo(E[3]))  // A{14}

// T4 activities (A{15}-A{18} in MATLAB)
A.add(Activity(model, "A1", Exp.fitMean(7.0)).on(T[3]).boundTo(E[4]).repliesTo(E[4]))  // A{15}
A.add(Activity(model, "A2", Exp.fitMean(4.0)).on(T[3]).boundTo(E[5]))  // A{16}
A.add(Activity(model, "A3", Exp.fitMean(5.0)).on(T[3]).boundTo(E[6]).repliesTo(E[6]))  // A{17}
A.add(Activity(model, "A2_Res_Empty", Immediate()).on(T[3]).synchCall(E[1], 1).repliesTo(E[5]))  // A{18}

// T5 activities (A{19}-A{22} in MATLAB)
A.add(Activity(model, "A4", Exp.fitMean(8.0)).on(T[4]).boundTo(E[7]).repliesTo(E[7]))  // A{19}
A.add(Activity(model, "A5", Exp.fitMean(4.0)).on(T[4]).boundTo(E[8]))  // A{20}
A.add(Activity(model, "A6", Exp.fitMean(6.0)).on(T[4]).boundTo(E[9]).repliesTo(E[9]))  // A{21}
A.add(Activity(model, "A5_Res_Empty", Immediate()).on(T[4]).synchCall(E[2], 1).repliesTo(E[8]))  // A{22}

// T6 activities (A{23}-A{25} in MATLAB)
A.add(Activity(model, "A7", Exp.fitMean(6.0)).on(T[5]).boundTo(E[10]).repliesTo(E[10]))  // A{23}
A.add(Activity(model, "A8", Exp.fitMean(8.0)).on(T[5]).boundTo(E[11]).repliesTo(E[11]))  // A{24}
A.add(Activity(model, "A11", Exp.fitMean(4.0)).on(T[5]).boundTo(E[12]).repliesTo(E[12]))  // A{25}

// T7 activities (A{26}-A{29} in MATLAB)
A.add(Activity(model, "A9", Exp.fitMean(4.0)).on(T[6]).boundTo(E[13]))  // A{26}
A.add(Activity(model, "A10", Exp.fitMean(6.0)).on(T[6]).boundTo(E[14]).repliesTo(E[14]))  // A{27}
A.add(Activity(model, "A12", Exp.fitMean(8.0)).on(T[6]).boundTo(E[15]).repliesTo(E[15]))  // A{28}
A.add(Activity(model, "A9_Res_Empty", Immediate()).on(T[6]).synchCall(E[3], 1).repliesTo(E[13]))  // A{29}

In [ ]:
// Add precedences - matching MATLAB lines 70-81
T[0].addPrecedence(ActivityPrecedence.Serial(A[0], A[1]))  // Line 70: Serial(A{1}, A{2})
T[0].addPrecedence(ActivityPrecedence.Serial(A[2], A[3]))  // Line 71: Serial(A{3}, A{4})
T[1].addPrecedence(ActivityPrecedence.Serial(A[6], A[7]))  // Line 72: Serial(A{7}, A{8})
T[1].addPrecedence(ActivityPrecedence.Serial(A[9], A[12]))  // Line 73: Serial(A{10}, A{13})
T[3].addPrecedence(ActivityPrecedence.Serial(A[15], A[17]))  // Line 74: Serial(A{16}, A{18})
T[4].addPrecedence(ActivityPrecedence.Serial(A[19], A[21]))  // Line 75: Serial(A{20}, A{22})
T[6].addPrecedence(ActivityPrecedence.Serial(A[25], A[28]))  // Line 76: Serial(A{26}, A{29})
T[0].addPrecedence(ActivityPrecedence.OrFork(A[1], listOf(A[4], A[5]), listOf(0.6, 0.4)))  // Line 77: OrFork(A{2}, {A{5}, A{6}}, [0.6,0.4])
T[1].addPrecedence(ActivityPrecedence.AndFork(A[7], listOf(A[8], A[9])))  // Line 78: AndFork(A{8}, {A{9}, A{10}})
T[0].addPrecedence(ActivityPrecedence.OrJoin(listOf(A[4], A[5]), A[2]))  // Line 79: OrJoin({A{5}, A{6}}, A{3})
T[1].addPrecedence(ActivityPrecedence.AndJoin(listOf(A[8], A[12]), A[10]))  // Line 80: AndJoin({A{9}, A{13}}, A{11})

In [ ]:
println("This example illustrates the solution of a complex layered queueing network extracted from a BPMN model.")

// Solve with LQNS solver
val options = LQNS.defaultOptions()
options.keep = true  // uncomment to keep the intermediate XML files generated while translating the model to LQNS

if (LQNS.isAvailable()) {
    val solver = LQNS(model, options)
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error: ${e.message}")
    }
} else {
    println("LQNS solver is not available.")
}

// Note: The MATLAB version comments out the LN with MVA option

This complex BPMN-derived layered queueing network demonstrates:

1. **BPMN Integration**:
   - Model extracted from Business Process Model and Notation
   - Complex workflow patterns translated to queueing constructs
   - Real-world business process modeling

2. **Multi-Layer Architecture**:
   - **7 Processors**: R1, R2, R3, R1A, R1B, R2A, R2B
   - **7 Tasks**: Each task running on specific processors
   - **16 Entries**: Service access points for synchronous calls
   - **29 Activities**: Detailed workflow steps

3. **Complex Precedence Patterns**:
   - **Serial precedences**: Sequential activity execution
   - **OrFork/OrJoin**: Probabilistic branching (60%/40% split)
   - **AndFork/AndJoin**: Parallel execution with synchronization
   - **Mixed patterns**: Combination of sequential and parallel flows

4. **Synchronous Communication**:
   - **synchCall()**: Synchronous service requests between layers
   - **repliesTo()**: Response patterns for request-reply interactions
   - **boundTo()**: Entry point bindings for activities

5. **Resource Hierarchy**:
   - **R1**: 100 threads (reference task with think time)
   - **R2**: Infinite capacity (unlimited parallelism)
   - **R3**: 2 threads (bottleneck resource)
   - **R1A/R1B/R2A/R2B**: Various capacities (3-7 threads)

6. **Service Time Distribution**:
   - **Reference task**: 20-unit think time
   - **Activities**: 4-10 unit exponential service times
   - **Empty activities**: Immediate (no processing delay)

The model represents:
- **Enterprise applications**: Multi-tier service architectures
- **Workflow systems**: Complex business process execution
- **Service-oriented architectures**: Layered service interactions
- **Microservices**: Distributed system communication patterns

Key insights:
- **BPMN translation**: Business processes can be systematically converted to performance models
- **Layered performance**: Multi-tier architectures require specialized solvers (LQNS)
- **Complexity management**: Systematic naming and organization essential for large models

**Analysis Method**: Uses LQNS (Layered Queueing Network Solver) which is specifically designed for multi-tier systems with synchronous communication patterns, unlike traditional queueing network solvers that focus on asynchronous message passing.